In [17]:
!pip install --upgrade jinja2

In [18]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-z_q_vb3z
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-z_q_vb3z
  Resolved https://github.com/huggingface/transformers.git to commit 0548af54ccc81d31e6264bb394e74c89c477518d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [19]:
import torch
from transformers import pipeline


In [20]:
!pip install gradio_client

In [21]:
!pip install jinja2==3.1.3

In [22]:
!pip install pyecharts

In [23]:
import json
#from dotenv import load_dotenv
from datetime import datetime, timedelta
#import openai
import os
from gradio_client import Client
from pyecharts import *
#from plot import plot_line_data_to_html, plot_accident_data_to_html, plot_gpt_data_to_html
#PLOT GPT generated top station data
import re
import matplotlib.pyplot as plt
from pyecharts.charts import Bar, Page, Pie, Grid
from pyecharts import options as opts


#load_dotenv()

# Define the function to plot the gpt data
def plot_gpt_data_to_html(filtered_gpt_data):
    data = filtered_gpt_data

    # Updated pattern to handle both data formats
    pattern = r'(\w[\w\s,]*[^\s])\s\((\d+)\)|\(([\w\s,]+), (\d+)\)'
    matches = re.findall(pattern, data)

    # Extracting station names and frequencies considering both matching groups from the updated pattern
    stations = [match[0] if match[2] == '' else match[2] for match in matches]
    frequencies = [int(match[1]) if match[3] == '' else int(match[3]) for match in matches]

    bar_chart = (
        Bar()
        .add_xaxis(stations)
        .add_yaxis("Frequenz", frequencies, tooltip_opts=opts.TooltipOpts(is_show=True, trigger="axis", axis_pointer_type="cross"))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Top 10 Betroffenen Stationen"),
            xaxis_opts=opts.AxisOpts(
                name="Stationen",  # X-Axis title
                axislabel_opts=opts.LabelOpts(
                    rotate=45,   # Try 45-degree rotation
                    font_size=8  # Further reduce font size
                )
            ),
            yaxis_opts=opts.AxisOpts(
                name="Frequenz",  # Y-Axis title
            )
        )
    )

    return bar_chart.render_notebook()

# Configure the OpenAI API client
#openai.api_type = os.getenv("OPENAI_API_TYPE")
#openai.api_version = os.getenv("OPENAI_API_VERSION")
#openai.api_base = os.getenv("OPENAI_API_BASE")  # Your Azure OpenAI resource's endpoint value.
#openai.api_key = os.getenv("OPENAI_API_KEY")

# Define the function to generate the response
#def generate_response(resulted_data):
#    response = openai.ChatCompletion.create(
#        model="gpt-4",
        #engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
#        temperature = 0.4,
#        messages=[
#            {"role": "system", "content": "Du bist ein Analyse-Chatbot. Aus den bereitgestellten JSON-Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-JSON-Daten:"+ resulted_data},
#            {"role": "user", "content": "Im JSON-Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
#            }
#        ]
#    )
#    return response['choices'][0]['message']['content']

def generate_response(resulted_data):


    response = openai.ChatCompletion.create(
    model="gpt-4-0314",
    # engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
    temperature=0.7,
    messages=[
      {"role": "user",
       "content": "You are an analyst. From the data provided, you answer user questions to create statistics based on user input. This is the context list data:" + resulted_data + "In the Vienna Line data context, affected lines are under title and under 'Description' lists affected stations. Which 10 stations are most frequently affected? Only output in this format: (station name, total number of incidents). For example: (Rotkreuzplatz, 10)."
      }
    ]
  )
    return response['choices'][0]['message']['content']



def generate_response_tinyllama(resulted_data):
    client = Client("https://tinyllama-tinyllama-chat.hf.space/--replicas/q15sq/")
    result = client.predict("You are an analyst. From the data provided, you answer user questions to create statistics based on user input. This is the context list data:" + resulted_data + "In the Vienna Line data context, affected lines are under title and under 'Description' lists affected stations. Which 10 stations are most frequently affected? Only output in this format: (station name, total number of incidents). For example: (Rotkreuzplatz, 10).",	 # str  in 'Message' Textbox component
                            api_name="/chat")

    return result

def generate_response_tinyllama2(resulted_data, temp):

    pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16)
    # We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
    messages = [
    {
        "role": "system",
        "content": "You are an analyst. From the data provided, you answer user questions to create statistics based on user input. This is the context list data:" + resulted_data + "In the Vienna Line data context, affected lines are under title and under 'Description' lists affected stations. Which 10 stations are most frequently affected? Only output in this format: (station name, total number of incidents). For example: (Rotkreuzplatz, 10)."
    }
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=512, do_sample=True, temperature=temp, top_k=50, top_p=0.95)

    #print(outputs[0]["generated_text"])

    #client = Client("https://tinyllama-tinyllama-chat.hf.space/--replicas/q15sq/")
    #result = client.predict("You are an analyst. From the data provided, you answer user questions to create statistics based on user input. This is the context list data:" + resulted_data + "In the Vienna Line data context, affected lines are under title and under 'Description' lists affected stations. Which 10 stations are most frequently affected? Only output in this format: (station name, total number of incidents). For example: (Rotkreuzplatz, 10).",	 # str  in 'Message' Textbox component
    #                        api_name="/chat")

    return outputs[0]["generated_text"]


# Define the function to filter the accident data
def get_incident_data(data, start_time, end_time, temp):
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)

    #gpt filter
    filtered_incidents = [
        {
            'title': entry['title'],
            'description': entry['description']
        }
        for entry in data['incidents']
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime
    ]

    resulted_data = json.dumps(filtered_incidents)

    print(resulted_data)

    #filtered_gpt_data = ''
    filtered_gpt_data = generate_response_tinyllama2(resulted_data, temp)

    return filtered_gpt_data

# Define the function to generate the date ranges
def get_date_ranges():
    # Initialize the end date to today's date at 00:00:00
    end_time = datetime.strptime('2015-08-16 00:00:00.00000','%Y-%m-%d %H:%M:%S.%f')
    # Define the time interval (1 week = 7 days)
    interval = timedelta(days=1)
    date_ranges = []
    # Generate the ranges
    for _ in range(3):
        start_time = end_time - interval
        date_ranges.append((start_time, end_time))
        end_time = start_time
    return date_ranges


# Define the function to generate the Subway_line data
def get_line_data(date_ranges):
    with open('hypo_2015-8-15.json', 'r') as file:
        data = json.load(file)

    #top 10 line data
    lines_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filetered_top_10_line_data = get_top10_lines(data, start_str, end_str)
        #print("Line data from data analysis:")
        lines_data_list.append(filetered_top_10_line_data)
    print(lines_data_list)
    return plot_line_data_to_html(lines_data_list)


# Define the function to generate the accident data
def get_accident_data(date_ranges):
    with open('hypo_2015-8-15.json', 'r') as file:
        data = json.load(file)

    #accident time sum daata
    accident_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filtered_accident_time_data = get_accident_time_sum(data, start_str, end_str)
        #print("Filtered Accident time data")
        accident_data_list.append(filtered_accident_time_data)
    print(accident_data_list)
    return plot_accident_data_to_html(accident_data_list)


# Define the function to generate the gpt data
def get_gpt_data(date_ranges, temp):
    # Use the list of tuples in a for loop
    data = read_data()
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')

        print(f"Start: {start_str}, End: {end_str}")
        filtered_gpt_data = get_incident_data(data, start_str, end_str, temp)

        print("Filtered TinyLLAM station data:")
        print(filtered_gpt_data)
        break
    return plot_gpt_data_to_html(filtered_gpt_data)

# Define the function to read the data
def read_data():
    with open('hypo_2015-8-15.json', 'r') as file:
        data = json.load(file)
    return data
#get_line_data(get_date_ranges())
#get_accident_data(get_date_ranges())
#get_gpt_data(get_date_ranges())


In [24]:
get_gpt_data(get_date_ranges(),0.7)

Start: 2015-08-15 00:00:00, End: 2015-08-16 00:00:00
[{"title": "5 : Falschparker", "description": "Due to a parking offence in the Blindengasse area, line 5 only runs between Praterstern S U and Josefst\u00c3\u00a4dter Stra\u00c3\u009fe U. Alternatively, please use line U6 . "}, {"title": "38 : Traffic accident", "description": "Due to a traffic accident in the area of Billrothstra\u00c3\u009fe, line 38 only runs between Schottentor U and Hohe Warte. "}, {"title": "40 , 41 : Fire brigade operation", "description": "Due to a fire brigade operation in the W\u00c3\u00a4hringer Stra\u00c3\u009fe Martinstra\u00c3\u009fe area, lines 40 and 41 are being diverted between W\u00c3\u00a4hringer Stra\u00c3\u009fe, Volksoper U and Gersthof S \u00c3\u00bcvia lines 42+9. "}, {"title": "49 : Traffic accident", "description": "Due to a traffic accident in the Westbahnstra\u00c3\u009fe area, line 49 only runs between Baumgarten and Urban-Loritz-Platz."}, {"title": "1 : Wrong-way parker", "description":

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Filtered TinyLLAM station data:
<|system|>
You are an analyst. From the data provided, you answer user questions to create statistics based on user input. This is the context list data:[{"title": "5 : Falschparker", "description": "Due to a parking offence in the Blindengasse area, line 5 only runs between Praterstern S U and Josefst\u00c3\u00a4dter Stra\u00c3\u009fe U. Alternatively, please use line U6 . "}, {"title": "38 : Traffic accident", "description": "Due to a traffic accident in the area of Billrothstra\u00c3\u009fe, line 38 only runs between Schottentor U and Hohe Warte. "}, {"title": "40 , 41 : Fire brigade operation", "description": "Due to a fire brigade operation in the W\u00c3\u00a4hringer Stra\u00c3\u009fe Martinstra\u00c3\u009fe area, lines 40 and 41 are being diverted between W\u00c3\u00a4hringer Stra\u00c3\u009fe, Volksoper U and Gersthof S \u00c3\u00bcvia lines 42+9. "}, {"title": "49 : Traffic accident", "description": "Due to a traffic accident in the Westbahnstra

In [25]:
get_gpt_data(get_date_ranges(),0.5)

Start: 2015-08-15 00:00:00, End: 2015-08-16 00:00:00
[{"title": "5 : Falschparker", "description": "Due to a parking offence in the Blindengasse area, line 5 only runs between Praterstern S U and Josefst\u00c3\u00a4dter Stra\u00c3\u009fe U. Alternatively, please use line U6 . "}, {"title": "38 : Traffic accident", "description": "Due to a traffic accident in the area of Billrothstra\u00c3\u009fe, line 38 only runs between Schottentor U and Hohe Warte. "}, {"title": "40 , 41 : Fire brigade operation", "description": "Due to a fire brigade operation in the W\u00c3\u00a4hringer Stra\u00c3\u009fe Martinstra\u00c3\u009fe area, lines 40 and 41 are being diverted between W\u00c3\u00a4hringer Stra\u00c3\u009fe, Volksoper U and Gersthof S \u00c3\u00bcvia lines 42+9. "}, {"title": "49 : Traffic accident", "description": "Due to a traffic accident in the Westbahnstra\u00c3\u009fe area, line 49 only runs between Baumgarten and Urban-Loritz-Platz."}, {"title": "1 : Wrong-way parker", "description":

In [26]:
get_gpt_data(get_date_ranges(),0.4)

Start: 2015-08-15 00:00:00, End: 2015-08-16 00:00:00
[{"title": "5 : Falschparker", "description": "Due to a parking offence in the Blindengasse area, line 5 only runs between Praterstern S U and Josefst\u00c3\u00a4dter Stra\u00c3\u009fe U. Alternatively, please use line U6 . "}, {"title": "38 : Traffic accident", "description": "Due to a traffic accident in the area of Billrothstra\u00c3\u009fe, line 38 only runs between Schottentor U and Hohe Warte. "}, {"title": "40 , 41 : Fire brigade operation", "description": "Due to a fire brigade operation in the W\u00c3\u00a4hringer Stra\u00c3\u009fe Martinstra\u00c3\u009fe area, lines 40 and 41 are being diverted between W\u00c3\u00a4hringer Stra\u00c3\u009fe, Volksoper U and Gersthof S \u00c3\u00bcvia lines 42+9. "}, {"title": "49 : Traffic accident", "description": "Due to a traffic accident in the Westbahnstra\u00c3\u009fe area, line 49 only runs between Baumgarten and Urban-Loritz-Platz."}, {"title": "1 : Wrong-way parker", "description":

In [27]:
get_gpt_data(get_date_ranges(),0.3)

Start: 2015-08-15 00:00:00, End: 2015-08-16 00:00:00
[{"title": "5 : Falschparker", "description": "Due to a parking offence in the Blindengasse area, line 5 only runs between Praterstern S U and Josefst\u00c3\u00a4dter Stra\u00c3\u009fe U. Alternatively, please use line U6 . "}, {"title": "38 : Traffic accident", "description": "Due to a traffic accident in the area of Billrothstra\u00c3\u009fe, line 38 only runs between Schottentor U and Hohe Warte. "}, {"title": "40 , 41 : Fire brigade operation", "description": "Due to a fire brigade operation in the W\u00c3\u00a4hringer Stra\u00c3\u009fe Martinstra\u00c3\u009fe area, lines 40 and 41 are being diverted between W\u00c3\u00a4hringer Stra\u00c3\u009fe, Volksoper U and Gersthof S \u00c3\u00bcvia lines 42+9. "}, {"title": "49 : Traffic accident", "description": "Due to a traffic accident in the Westbahnstra\u00c3\u009fe area, line 49 only runs between Baumgarten and Urban-Loritz-Platz."}, {"title": "1 : Wrong-way parker", "description":

In [28]:
get_gpt_data(get_date_ranges(),0.2)

Start: 2015-08-15 00:00:00, End: 2015-08-16 00:00:00
[{"title": "5 : Falschparker", "description": "Due to a parking offence in the Blindengasse area, line 5 only runs between Praterstern S U and Josefst\u00c3\u00a4dter Stra\u00c3\u009fe U. Alternatively, please use line U6 . "}, {"title": "38 : Traffic accident", "description": "Due to a traffic accident in the area of Billrothstra\u00c3\u009fe, line 38 only runs between Schottentor U and Hohe Warte. "}, {"title": "40 , 41 : Fire brigade operation", "description": "Due to a fire brigade operation in the W\u00c3\u00a4hringer Stra\u00c3\u009fe Martinstra\u00c3\u009fe area, lines 40 and 41 are being diverted between W\u00c3\u00a4hringer Stra\u00c3\u009fe, Volksoper U and Gersthof S \u00c3\u00bcvia lines 42+9. "}, {"title": "49 : Traffic accident", "description": "Due to a traffic accident in the Westbahnstra\u00c3\u009fe area, line 49 only runs between Baumgarten and Urban-Loritz-Platz."}, {"title": "1 : Wrong-way parker", "description":

In [29]:
get_gpt_data(get_date_ranges(),0.1)

Start: 2015-08-15 00:00:00, End: 2015-08-16 00:00:00
[{"title": "5 : Falschparker", "description": "Due to a parking offence in the Blindengasse area, line 5 only runs between Praterstern S U and Josefst\u00c3\u00a4dter Stra\u00c3\u009fe U. Alternatively, please use line U6 . "}, {"title": "38 : Traffic accident", "description": "Due to a traffic accident in the area of Billrothstra\u00c3\u009fe, line 38 only runs between Schottentor U and Hohe Warte. "}, {"title": "40 , 41 : Fire brigade operation", "description": "Due to a fire brigade operation in the W\u00c3\u00a4hringer Stra\u00c3\u009fe Martinstra\u00c3\u009fe area, lines 40 and 41 are being diverted between W\u00c3\u00a4hringer Stra\u00c3\u009fe, Volksoper U and Gersthof S \u00c3\u00bcvia lines 42+9. "}, {"title": "49 : Traffic accident", "description": "Due to a traffic accident in the Westbahnstra\u00c3\u009fe area, line 49 only runs between Baumgarten and Urban-Loritz-Platz."}, {"title": "1 : Wrong-way parker", "description":

In [30]:
get_gpt_data(get_date_ranges(),0.8)

Start: 2015-08-15 00:00:00, End: 2015-08-16 00:00:00
[{"title": "5 : Falschparker", "description": "Due to a parking offence in the Blindengasse area, line 5 only runs between Praterstern S U and Josefst\u00c3\u00a4dter Stra\u00c3\u009fe U. Alternatively, please use line U6 . "}, {"title": "38 : Traffic accident", "description": "Due to a traffic accident in the area of Billrothstra\u00c3\u009fe, line 38 only runs between Schottentor U and Hohe Warte. "}, {"title": "40 , 41 : Fire brigade operation", "description": "Due to a fire brigade operation in the W\u00c3\u00a4hringer Stra\u00c3\u009fe Martinstra\u00c3\u009fe area, lines 40 and 41 are being diverted between W\u00c3\u00a4hringer Stra\u00c3\u009fe, Volksoper U and Gersthof S \u00c3\u00bcvia lines 42+9. "}, {"title": "49 : Traffic accident", "description": "Due to a traffic accident in the Westbahnstra\u00c3\u009fe area, line 49 only runs between Baumgarten and Urban-Loritz-Platz."}, {"title": "1 : Wrong-way parker", "description":

In [31]:
get_gpt_data(get_date_ranges(),0.9)

Start: 2015-08-15 00:00:00, End: 2015-08-16 00:00:00
[{"title": "5 : Falschparker", "description": "Due to a parking offence in the Blindengasse area, line 5 only runs between Praterstern S U and Josefst\u00c3\u00a4dter Stra\u00c3\u009fe U. Alternatively, please use line U6 . "}, {"title": "38 : Traffic accident", "description": "Due to a traffic accident in the area of Billrothstra\u00c3\u009fe, line 38 only runs between Schottentor U and Hohe Warte. "}, {"title": "40 , 41 : Fire brigade operation", "description": "Due to a fire brigade operation in the W\u00c3\u00a4hringer Stra\u00c3\u009fe Martinstra\u00c3\u009fe area, lines 40 and 41 are being diverted between W\u00c3\u00a4hringer Stra\u00c3\u009fe, Volksoper U and Gersthof S \u00c3\u00bcvia lines 42+9. "}, {"title": "49 : Traffic accident", "description": "Due to a traffic accident in the Westbahnstra\u00c3\u009fe area, line 49 only runs between Baumgarten and Urban-Loritz-Platz."}, {"title": "1 : Wrong-way parker", "description":

In [32]:
get_gpt_data(get_date_ranges(),0.6)

Start: 2015-08-15 00:00:00, End: 2015-08-16 00:00:00
[{"title": "5 : Falschparker", "description": "Due to a parking offence in the Blindengasse area, line 5 only runs between Praterstern S U and Josefst\u00c3\u00a4dter Stra\u00c3\u009fe U. Alternatively, please use line U6 . "}, {"title": "38 : Traffic accident", "description": "Due to a traffic accident in the area of Billrothstra\u00c3\u009fe, line 38 only runs between Schottentor U and Hohe Warte. "}, {"title": "40 , 41 : Fire brigade operation", "description": "Due to a fire brigade operation in the W\u00c3\u00a4hringer Stra\u00c3\u009fe Martinstra\u00c3\u009fe area, lines 40 and 41 are being diverted between W\u00c3\u00a4hringer Stra\u00c3\u009fe, Volksoper U and Gersthof S \u00c3\u00bcvia lines 42+9. "}, {"title": "49 : Traffic accident", "description": "Due to a traffic accident in the Westbahnstra\u00c3\u009fe area, line 49 only runs between Baumgarten and Urban-Loritz-Platz."}, {"title": "1 : Wrong-way parker", "description":

In [33]:
get_gpt_data(get_date_ranges(),1)

Start: 2015-08-15 00:00:00, End: 2015-08-16 00:00:00
[{"title": "5 : Falschparker", "description": "Due to a parking offence in the Blindengasse area, line 5 only runs between Praterstern S U and Josefst\u00c3\u00a4dter Stra\u00c3\u009fe U. Alternatively, please use line U6 . "}, {"title": "38 : Traffic accident", "description": "Due to a traffic accident in the area of Billrothstra\u00c3\u009fe, line 38 only runs between Schottentor U and Hohe Warte. "}, {"title": "40 , 41 : Fire brigade operation", "description": "Due to a fire brigade operation in the W\u00c3\u00a4hringer Stra\u00c3\u009fe Martinstra\u00c3\u009fe area, lines 40 and 41 are being diverted between W\u00c3\u00a4hringer Stra\u00c3\u009fe, Volksoper U and Gersthof S \u00c3\u00bcvia lines 42+9. "}, {"title": "49 : Traffic accident", "description": "Due to a traffic accident in the Westbahnstra\u00c3\u009fe area, line 49 only runs between Baumgarten and Urban-Loritz-Platz."}, {"title": "1 : Wrong-way parker", "description":